여러개의 특성을 사용한 선형 회귀를 ***다중 회귀***라고함
- 특성이 1개면 직선을, 특성이 2개면 평면을 학습함

기존의 특성을 사용하여 새로운 특성을 뽑아내는 작업을 ***특성 공학***이라고 함

특성이 늘어났기 때문에 복붙보단 데이터를 다운로드 -> pandas사용

dataframe은 pandas의 핵심 데이터 구조

dataframe은 넘파이 배열보다 더 많은 기능 제공, 넘파이 배열로 쉽게 변경 가능

In [ ]:
# 데이터 준비
# https://bit.ly/perch_csv_data 에 접속하여 데이터 다운로드

# pandas 호출
import pandas as pd

# 데이터 다운로드하여 df로 저장
df = pd.read_csv('https://bit.ly/perch_csv_data')

# df를 numpy배열로 변경
perch_full = df.to_numpy()

print(perch_full)

In [ ]:
# 타깃 데이터는 이전과 동일한 방식으로 준비
# https://bit.ly/perch_data 에서 복붙

# numpy 호출
import numpy as np

# 타깃 데이터 준비
perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])

In [ ]:
# perch_full과 perch_weight을 훈련 세트와 테스트 세트로 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    perch_full, perch_weight, random_state=42)

사이킷런에서는 특성을 만들거나 전처리하기 위한 다양한 클래스 제공

이러한 클래스를 ***변환기***라고 부름

변환기 클래스는 모두 fit(), score() 메서드 제공

In [ ]:
# PolynomialFeatures 클래스 사용
# sklearn.preprocessing 패키지에 포함
from sklearn.preprocessing import PolynomialFeatures

# 변환기 객체 생성
poly = PolynomialFeatures()

poly.fit([[2,3]])
print(poly.transform([[2,3]]))

PolynomialFeatures는 기본적으로 각 특성의 제곱한 항을 추가하고 특성끼리 서로 곱한 항을 추가

$
무게 = a * 길이 + b * 높이 + c * 두께 + 1
$

선형 방정식의 절편을 항상 1인 특성과 곱해지는 계수라고 볼 수 있음

이렇게 놓고 보면 특성은 (길이,높이,두께.1)이 된다

사이킷런의 선형 모델은 자동으로 절편을 추가 하므로 특성을 만들 필요가 없다

include_bias=False로 지정하여 절편 없이 생성 가능(지정하지 않아도 사이킷런 모델은 자동으로 특성에 추가된 절편 항을 무시하므로 필수는 아님)

In [ ]:
#include_bias=False 추가하여 다시 생성
poly = PolynomialFeatures(include_bias=False)
poly.fit([[2,3]])
print(poly.transform([[2,3]]))
# 절편을 위한 항이 제거되고 특성의 제공과 특성끼리 곱한 항만 추가

In [ ]:
# 이 방식으로 train_input에 적용
# train_input을 변환한 데이터를 train_poly에 저장하고 배열 크기 확인
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
print(train_poly.shape)

In [ ]:
# get_features_names_out()메서드 호출하여 특성 확인
poly.get_feature_names_out()

In [ ]:
# 테스트 세트 변환
test_poly = poly.transform(test_input)

In [ ]:
# LinearRegression 클래스 호출하고 train_poly사용하여 모델 훈련
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_poly, train_target)

In [ ]:
# 점수 출력
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

In [ ]:
# PolynomialFeatures클래스의 degree 매개변수로 고차항의 최대 차수 지정가능
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
print(train_poly.shape)

In [ ]:
# 선형회귀 모델로 다시 학습
lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

***규제***는 머신러닝 모델이 훈련 세트를 너무 과도하게 학습하지 못하도록 훼방하는 것을 말함

즉, 모델이 훈련 세트에 과대적합되지 않도록 만드는 것
- 선형 회귀 모델의 경우 특성에 곱해지는 계수(또는 기울기)의 크기를 작게 만드는 일

일반적으로 선형 회귀 모델에 규제를 적용할 때 계수 값의 크기가 서로 많이 다르면 공정하게 제어되지 않을 수 있음
- 규제 전에 먼저 정규화가 필요
- 사이킷런에서 제공하는 StandardScaler 클래스 사용


In [ ]:
# StandardScaler 호출
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_poly)

# 꼭 훈련 세트로 학습한 변환기를 사용해 테스트 세트까지 변환해야함
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

선형 회귀 모델에 규제를 추가한 모델을 ***릿지***와 ***라쏘***라고 부름
- 릿지는 계수를 제곱한 값을 기준으로 규제 적용
- 라쏘는 계수의 절댓값을 기준으로 규제 적용
- 일반적으로 릿지를 조금 더 선호

In [ ]:
# 릿지 회귀
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

릿지와 라쏘 모델을 사용할 때 규제의 양을 임의로 조절 가능
- 모델 객체를 만들 때 alpha 매개변수로 규제의 강도 조절
- alpha값이 크면 규제 강도가 세지므로 계수 값을 더 줄이고 조금 더 과소적합되도록 유도
- alpha값이 작으면 계수를 줄이는 역할이 줄어들고 선형 회귀 모델과 유사해지므로 과대적합할 가능성이 큼
- alpha 값은 사전에 사람이 지정해야 하는 값
    - 이러한 머신러닝 모델이 학습할 수 없고 사람이 알려줘야 하는 파라미터를 ***하이퍼파라미터***라고 함

In [ ]:
# 적절한 alpha 값을 찾는 법은 alpha 값에 대한 R^2 값의 그래프를 그려 보는 것
# 훈련 세트와 테스트 세트의 점수가 가장 가까운 지점이 최적의 alpha
import matplotlib.pyplot as plt

# alpha값을 바꿀 때마다 score() 메서드의 결과를 저장할 리스트 생성
train_score = []
test_score = []

In [ ]:
# alpha값을 0.001에서 100까지 10배씩 늘려가며 릿지 회귀 모델 훈련
# 훈련 세트와 테스트 세트의 점수를 파이썬 리스트에 저장
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    # 릿지 모델 생성
    ridge = Ridge(alpha=alpha)
    # 릿지 모델 훈련
    ridge.fit(train_scaled, train_target)
    # 훈련 점수와 테스트 점수를 저장
    train_score.append(ridge.score(train_scaled, train_target))
    test_score.append(ridge.score(test_scaled, test_target))

In [ ]:
# np.log10 로그 함수 사용하여 6개의 값을 동일한 간격으로 나타냄
plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()

# 두 그래프가 가장 가까운 지점인 -1, 즉 10^-1=0.1이 최적의 alpha값

In [ ]:
# alpha=0.1로 최종 모델을 훈련
ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

In [ ]:
# 라쏘 회귀
# Ridge 클래스를 Lasso 클래스로 바꾸는 것
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target))

In [ ]:
# 테스트 점수 확인
print(lasso.score(test_scaled, test_target))

In [ ]:
# 라쏘 모델도 alpha 값을 바꾸어 가면 최적 값 도출
train_score = []
test_score = []
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    # 라쏘 모델 생성
    lasso = Lasso(alpha=alpha, max_iter=10000)
    # 라쏘 모델 훈련
    lasso.fit(train_scaled, train_target)
    # 훈련 점수와 테스트 점수를 저장
    train_score.append(lasso.score(train_scaled, train_target))
    test_score.append(lasso.score(test_scaled, test_target))

In [ ]:
# train_score와 test_score를 사용해 그래프 플롯
plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()

# 최적의 alpha값은 1, 즉 10^1=10

In [ ]:
# 최적 alpha값으로 최종 모델 훈련
lasso = Lasso(alpha=10)
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

In [ ]:
# lasso모델의 계수는 coef_ 속성에 저장되어 있음
# 이 중에 0인 것 찾기
print(np.sum(lasso.coef_ == 0))

# 총 55개의 특성 중 15개만 사용
# 라쏘모델은 유용한 특성을 골라내는 용도로도 사용 가능